In [ ]:
import os
import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input,BatchNormalization
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.merge import average
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from imgaug import augmenters as iaa
from random import randint

In [ ]:
im_width = 128
im_height = 128
im_chan =3
path_train = '../input/train/'
path_test = '../input/test/'

In [ ]:
ids= ['1f1cc6b3a4','5b7c160d0d','6c40978ddf','7dfdf6eeb8','7e5a6e5013']
plt.figure(figsize=(20,10))
for j, img_name in enumerate(ids):
    q = j+1
    img = load_img('../input/train/images/' + img_name + '.png')
    img_mask = load_img('../input/train/masks/' + img_name + '.png')
    
    plt.subplot(1,2*(1+len(ids)),q*2-1)
    plt.imshow(img)
    plt.subplot(1,2*(1+len(ids)),q*2)
    plt.imshow(img_mask)
plt.show()

In [ ]:
train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]

In [ ]:
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)
    x = resize(x, (128, 128, 3), mode='constant', preserve_range=True)
    X_train[n] = x
    mask=load_img(path + '/masks/' + id_)
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,0]
    Y_train[n] = resize(mask, (128, 128, 1), mode='constant', preserve_range=True)

print('Done!')
print(X_train.shape,Y_train.shape)

In [ ]:
X_val=X_train[:1000]
X_train=X_train[1000:]
Y_val=Y_train[:1000]
Y_train=Y_train[1000:]
print(X_train.shape,X_val.shape,Y_train.shape,Y_val.shape)

In [ ]:
def generator(features, labels, batch_size):
    # create empty arrays to contain batch of features and labels
    features_return=features.copy()
    labels_return=labels.copy()
    batch_features = np.zeros((batch_size, 128, 128, 3),dtype=np.uint8)
    batch_labels = np.zeros((batch_size, 128,128,1),dtype=np.bool)
    aug=iaa.Sequential([iaa.OneOf([iaa.Noop(),
                                  iaa.Fliplr(0.5),
                                  iaa.Flipud(0.5)])])
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = np.random.randint(0, features_return.shape[0]-1)
            _aug=aug.to_deterministic()
            random_augmented_image =_aug.augment_image(features_return[index])
            random_augmented_label=_aug.augment_image(labels_return[index].astype(np.uint8))
            random_augmented_image=resize(random_augmented_image, (128, 128, 3), mode='constant', preserve_range=True)
            random_augmented_label=resize(random_augmented_label, (128, 128, 1), mode='constant', preserve_range=True)
            batch_features[i] = random_augmented_image
            batch_labels[i] = random_augmented_label
        yield batch_features, batch_labels

In [ ]:
i=0
for img_batch,mask_batch in generator(X_train,Y_train,8):
    for j,image in enumerate(img_batch):
        plt.imshow(image)
        plt.show()
        tmp=np.squeeze(mask_batch[j]).astype(np.float32)
        plt.imshow(np.dstack((tmp,tmp,tmp)))
        plt.show()
    i+=1
    if i>0:
        break

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (s)
c1=BatchNormalization()(c1)
c1 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c1)
c1=BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)
c2 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (p1)
c2=BatchNormalization()(c2)
c2 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c2)
c2=BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (p2)
c3=BatchNormalization()(c3)
c3 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c3)
c3=BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (p3)
c4=BatchNormalization()(c4)
c4 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c4)
c4=BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(1024, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (p4)
c5=BatchNormalization()(c5)
c5 = Conv2D(1024, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c5)
c5=BatchNormalization()(c5)

u6 = Conv2DTranspose(1024, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (u6)
c6=BatchNormalization()(c6)
c6 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c6)
c6=BatchNormalization()(c6)

u7 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (u7)
c7=BatchNormalization()(c7)
c7 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c7)
c7=BatchNormalization()(c7)

u8 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (u8)
c8=BatchNormalization()(c8)
c8 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c8)
c8=BatchNormalization()(c8)
u9 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (u9)
c9=BatchNormalization()(c9)
c9 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.00001)) (c9)
c9=BatchNormalization()(c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid',kernel_regularizer=l2(0.00001)) (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

In [ ]:
'''inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)
c_0_0=Conv2D(16,(3,3),activation="relu",padding="same")(s)
c_0_0=BatchNormalization()(c_0_0)
c_0_0=Conv2D(16,(3,3),activation="relu",padding="same")(c_0_0)
c_0_0=BatchNormalization()(c_0_0)
p_0_0=MaxPooling2D((2,2))(c_0_0)
c_1_0=Conv2D(32,(3,3),activation="relu",padding="same")(p_0_0)
c_1_0=BatchNormalization()(c_1_0)
c_1_0=Conv2D(32,(3,3),activation="relu",padding="same")(c_1_0)
c_1_0=BatchNormalization()(c_1_0)
p_1_0=MaxPooling2D((2,2))(c_1_0)
c_2_0=Conv2D(64,(3,3),activation="relu",padding="same")(p_1_0)
c_2_0=BatchNormalization()(c_2_0)
c_2_0=Conv2D(64,(3,3),activation="relu",padding="same")(c_2_0)
c_2_0=BatchNormalization()(c_2_0)
p_2_0=MaxPooling2D((2,2))(c_2_0)
c_3_0=Conv2D(128,(3,3),activation="relu",padding="same")(p_2_0)
c_3_0=BatchNormalization()(c_3_0)
c_3_0=Conv2D(128,(3,3),activation="relu",padding="same")(c_3_0)
c_3_0=BatchNormalization()(c_3_0)
p_3_0=MaxPooling2D((2,2))(c_3_0)
c_4_0=Conv2D(256,(3,3),activation="relu",padding="same")(p_3_0)
c_4_0=BatchNormalization()(c_4_0)
c_4_0=Conv2D(256,(3,3),activation="relu",padding="same")(c_4_0)
c_4_0=BatchNormalization()(c_4_0)
u_0_1=Conv2DTranspose(32,(2,2),strides=(2,2),padding="same")(c_1_0)
u_0_1=concatenate([u_0_1,c_0_0])
c_0_1=Conv2D(16,(3,3),activation="relu",padding="same")(u_0_1)
c_0_1=BatchNormalization()(c_0_1)
c_0_1=Conv2D(16,(3,3),activation="relu",padding="same")(c_0_1)
c_0_1=BatchNormalization()(c_0_1)
u_1_1=Conv2DTranspose(64,(2,2),strides=(2,2),padding="same")(c_2_0)
u_1_1=concatenate([u_1_1,c_1_0])
c_1_1=Conv2D(32,(3,3),activation="relu",padding="same")(u_1_1)
c_1_1=BatchNormalization()(c_1_1)
c_1_1=Conv2D(32,(3,3),activation="relu",padding="same")(c_1_1)
c_1_1=BatchNormalization()(c_1_1)
u_2_1=Conv2DTranspose(128,(2,2),strides=(2,2),padding="same")(c_3_0)
u_2_1=concatenate([u_2_1,c_2_0])
c_2_1=Conv2D(64,(3,3),activation="relu",padding="same")(u_2_1)
c_2_1=BatchNormalization()(c_2_1)
c_2_1=Conv2D(64,(3,3),activation="relu",padding="same")(c_2_1)
c_2_1=BatchNormalization()(c_2_1)
u_3_1=Conv2DTranspose(256,(2,2),strides=(2,2),padding="same")(c_4_0)
u_3_1=concatenate([u_3_1,c_3_0])
c_3_1=Conv2D(128,(3,3),activation="relu",padding="same")(u_3_1)
c_3_1=BatchNormalization()(c_3_1)
c_3_1=Conv2D(128,(3,3),activation="relu",padding="same")(c_3_1)
c_3_1=BatchNormalization()(c_3_1)
u_0_2=Conv2DTranspose(32,(2,2),strides=(2,2),padding="same")(c_1_1)
u_0_2=concatenate([u_0_2,c_0_1])
c_0_2=Conv2D(16,(3,3),activation="relu",padding="same")(u_0_2)
c_0_2=BatchNormalization()(c_0_2)
c_0_2=Conv2D(16,(3,3),activation="relu",padding="same")(c_0_2)
c_0_2=BatchNormalization()(c_0_2)
u_1_2=Conv2DTranspose(64,(2,2),strides=(2,2),padding="same")(c_2_1)
u_1_2=concatenate([u_1_2,c_1_1])
c_1_2=Conv2D(32,(3,3),activation="relu",padding="same")(u_1_2)
c_1_2=Conv2D(32,(3,3),activation="relu",padding="same")(c_1_2)
c_1_2=BatchNormalization()(c_1_2)
u_2_2=Conv2DTranspose(128,(2,2),strides=(2,2),padding="same")(c_3_1)
u_2_2=concatenate([u_2_2,c_2_1])
c_2_2=Conv2D(64,(3,3),activation="relu",padding="same")(u_2_2)
c_2_2=BatchNormalization()(c_2_2)
c_2_2=Conv2D(64,(3,3),activation="relu",padding="same")(c_2_2)
c_2_2=BatchNormalization()(c_2_2)
u_0_3=Conv2DTranspose(32,(2,2),strides=(2,2),padding="same")(c_1_2)
u_0_3=concatenate([u_0_3,c_0_2])
c_0_3=Conv2D(16,(3,3),activation="relu",padding="same")(u_0_3)
c_0_3=BatchNormalization()(c_0_3)
c_0_3=Conv2D(16,(3,3),activation="relu",padding="same")(c_0_3)
c_0_3=BatchNormalization()(c_0_3)
u_1_3=Conv2DTranspose(64,(2,2),strides=(2,2),padding="same")(c_2_2)
u_1_3=concatenate([u_1_3,c_1_2])
c_1_3=Conv2D(32,(3,3),activation="relu",padding="same")(u_1_3)
c_1_3=BatchNormalization()(c_1_3)
c_1_3=Conv2D(32,(3,3),activation="relu",padding="same")(c_1_3)
c_1_3=BatchNormalization()(c_1_3)
u_0_4=Conv2DTranspose(32,(2,2),strides=(2,2),padding="same")(c_1_3)
u_0_4=concatenate([u_0_4,c_0_3])
c_0_4=Conv2D(16,(3,3),activation="relu",padding="same")(u_0_4)
c_0_4=BatchNormalization()(c_0_4)
c_0_4=Conv2D(16,(3,3),activation="relu",padding="same")(c_0_4)
c_0_4=BatchNormalization()(c_0_4)
output1=Conv2D(1,(1,1),activation="sigmoid")(c_0_1)
output2=Conv2D(1,(1,1),activation="sigmoid")(c_0_2)
output3=Conv2D(1,(1,1),activation="sigmoid")(c_0_3)
output4=Conv2D(1,(1,1),activation="sigmoid")(c_0_4)
outputs=average([output1,output2,output3,output4])
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=[mean_iou])
model.summary()'''

In [ ]:
'''inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (s)
c1=BatchNormalization()(c1)
c1 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c1)
c1=BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)
c2 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (p1)
c2=BatchNormalization()(c2)
c2 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c2)
c2=BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (p2)
c3=BatchNormalization()(c3)
c3 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c3)
c3=BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (p3)
c4=BatchNormalization()(c4)
c4 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c4)
c4=BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (p4)
c5=BatchNormalization()(c5)
c5 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c5)
c5=BatchNormalization()(c5)

u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u6)
c6=BatchNormalization()(c6)
c6 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c6)
c6=BatchNormalization()(c6)

u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u7)
c7=BatchNormalization()(c7)
c7 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c7)
c7=BatchNormalization()(c7)

u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u8)
c8=BatchNormalization()(c8)
c8 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c8)
c8=BatchNormalization()(c8)
u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u9)
c9=BatchNormalization()(c9)
c9 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (c9)
c9=BatchNormalization()(c9)

outputs = Conv2D(3, (1, 1), activation='sigmoid',kernel_regularizer=l2(0.0001)) (c9)
inputs2=concatenate([s,outputs])
u2_c1 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (inputs2)
u2_c1=BatchNormalization()(u2_c1)
u2_c1 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c1)
u2_c1=BatchNormalization()(u2_c1)
u2_p1 = MaxPooling2D((2, 2)) (u2_c1)
u2_c2 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_p1)
u2_c2=BatchNormalization()(u2_c2)
u2_c2 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c2)
u2_c2=BatchNormalization()(u2_c2)
u2_p2 = MaxPooling2D((2, 2)) (u2_c2)

u2_c3 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_p2)
u2_c3=BatchNormalization()(u2_c3)
u2_c3 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c3)
u2_c3=BatchNormalization()(u2_c3)
u2_p3 = MaxPooling2D((2, 2)) (u2_c3)

u2_c4 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_p3)
u2_c4=BatchNormalization()(u2_c4)
u2_c4 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c4)
u2_c4=BatchNormalization()(u2_c4)
u2_p4 = MaxPooling2D(pool_size=(2, 2)) (u2_c4)

u2_c5 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_p4)
u2_c5=BatchNormalization()(u2_c5)
u2_c5 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c5)
u2_c5=BatchNormalization()(u2_c5)

u2_u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (u2_c5)
u2_u6 = concatenate([u2_u6, u2_c4])
u2_c6 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_u6)
u2_c6=BatchNormalization()(u2_c6)
u2_c6 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c6)
u2_c6=BatchNormalization()(u2_c6)

u2_u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u2_c6)
u2_u7 = concatenate([u2_u7, u2_c3])
u2_c7 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_u7)
u2_c7=BatchNormalization()(u2_c7)
u2_c7 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c7)
u2_c7=BatchNormalization()(u2_c7)

u2_u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (u2_c7)
u2_u8 = concatenate([u2_u8, u2_c2])
u2_c8 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_u8)
u2_c8=BatchNormalization()(u2_c8)
u2_c8 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c8)
u2_c8=BatchNormalization()(u2_c8)
u2_u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (u2_c8)
u2_u9 = concatenate([u2_u9, u2_c1], axis=3)
u2_c9 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_u9)
u2_c9=BatchNormalization()(u2_c9)
u2_c9 = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)) (u2_c9)
u2_c9=BatchNormalization()(u2_c9)

outputs2 = Conv2D(1, (1, 1), activation='sigmoid',kernel_regularizer=l2(0.0001)) (u2_c9)
model = Model(inputs=[inputs], outputs=[outputs2])
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=[mean_iou])
model.summary()'''

In [ ]:
earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint('model-tgs-salt-1.h5', verbose=1, save_best_only=True)
results = model.fit_generator(generator(X_train,Y_train,16),steps_per_epoch=188,epochs=100,validation_data=[X_val,Y_val],
                              callbacks=[earlystopper, checkpointer],verbose=1)

In [ ]:
loss=results.history["loss"]
val_loss=results.history["val_loss"]
mean_iou=results.history["mean_iou"]
val_mean_iou=results.history["val_mean_iou"]
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,"bo",label="Training Loss")
plt.plot(epochs,val_loss,"b",label="Validation Loss")
plt.title("Training and validation loss")
plt.legend()
plt.figure()
plt.plot(epochs,mean_iou,"bo",label="Training Mean_iou")
plt.plot(epochs,val_mean_iou,"b",label="Validation Mean_iou")
plt.title("Training and validation mean_iou")
plt.legend()
plt.show()

In [ ]:
ind_loss=np.argmin(val_loss)
print(val_mean_iou[ind_loss])

In [ ]:
X_test=np.zeros((len(test_ids),im_height,im_width,im_chan),dtype=np.uint8)
sizes_test=[]
print("Getting resizing test images...")
sys.stdout.flush()
for n,id_ in tqdm_notebook(enumerate(test_ids),total=len(test_ids)):
    path=path_test
    img=load_img(path+"/images/"+id_)
    x=img_to_array(img)
    sizes_test.append([x.shape[0],x.shape[1]])
    x=resize(x,(128,128,3),mode="constant",preserve_range=True)
    X_test[n]=x
print("Done")

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
model=load_model("model-tgs-salt-1.h5",custom_objects={"mean_iou":mean_iou})
preds_train=model.predict(X_train[:int(X_train.shape[0]*0.7)],verbose=1)
preds_val=model.predict(X_train[int(X_train.shape[0]*0.7):],verbose=1)
preds_test=model.predict(X_test,verbose=True)
preds_train_t=(preds_train>0.5).astype(np.uint8)
preds_val_t=(preds_val>0.5).astype(np.uint8)
preds_test_t=(preds_test>0.5).astype(np.uint8)

In [ ]:
pred_test_upsampled=[]
for i in tnrange(len(preds_test)):
    pred_test_upsampled.append(resize(np.squeeze(preds_test[i]),
                                               (sizes_test[i][0],sizes_test[i][1]),
                                               mode="constant",preserve_range=True))

In [ ]:
pred_test_upsampled[0].shape

In [ ]:
ix=random.randint(0,len(preds_train_t))
plt.imshow(X_train[ix])
plt.show()
tmp=np.squeeze(Y_train[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()
tmp=np.squeeze(preds_train_t[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn[:-4]:RLenc(np.round(pred_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
sub=pd.DataFrame.from_dict(pred_dict,orient="index")
sub.index.names=["id"]
sub.columns=["rle_mask"]
sub.to_csv("sub.csv")